# Batch Normalization

In [1]:
# Convolutional Neural Network (GOOD PIZZA  vs BURNT PIZZA)

# Building the CNN

#Importing Libraries
from tensorflow.keras.preprocessing.image import ImageDataGenerator
from tensorflow.keras.models import Sequential 
from tensorflow.keras.layers import Conv2D,MaxPooling2D,Activation,Dropout,Flatten,Dense, BatchNormalization
from tensorflow.keras import backend as K
import numpy as np
from tensorflow.keras.preprocessing import image


In [2]:
#loading data

image_width=150
image_height=150

# train_data_dir=r'PIZZA/Train'
# test_data_dir=r'PIZZA/Test'
# validation_data_dir=r'PIZZA/Val'

#train_sample=30
#validation_sample=25
test_sample=50

#epochs=30
batch_size=12
test_size=50

In [3]:
if K.image_data_format() == 'channels_first':
    input_shape= (3,image_width,image_height)
else :
    input_shape= (image_width,image_height,3)

In [4]:
# Data Augmentation

train_datagen= ImageDataGenerator(rotation_range=40,
                                  width_shift_range=0.2,
                                  height_shift_range=0.2,
                                  shear_range=0.2,
                                  zoom_range=0.2,
                                  rescale=1./255,
                                  horizontal_flip=True,
                                 fill_mode='nearest');

validation_datagen= ImageDataGenerator(rescale=1./255)
test_datagen=ImageDataGenerator(rescale=1./255)

In [6]:
train_genarator= train_datagen.flow_from_directory('data/pizza_ds/Train',
                                                   target_size=(image_width,image_height),
                                                   batch_size=batch_size,
                                                   class_mode='binary')


validation_genarator= validation_datagen.flow_from_directory('data/pizza_ds/Val',
                                                        target_size=(image_width,image_height),
                                                        batch_size=batch_size,
                                                        class_mode='binary')


test_genarator= test_datagen.flow_from_directory('data/pizza_ds/Test',
                                                 target_size=(image_width,image_height),
                                                 batch_size=test_size,
                                                 class_mode='binary')


Found 566 images belonging to 2 classes.
Found 221 images belonging to 2 classes.
Found 199 images belonging to 2 classes.


In [7]:
#build model

# Initialising the CNN

model = Sequential()
# Convolution
model.add(Conv2D(32,(3,3),input_shape=input_shape, activation='relu'))

# Pooling
model.add(MaxPooling2D(pool_size=(2,2)))
model.add(BatchNormalization())  # these batch normalized o/p's tend to converge faster

# Adding a second convolutional layer
model.add(Conv2D(32,(3,3),input_shape=input_shape, activation='relu'))
model.add(MaxPooling2D(pool_size=(2,2)))
model.add(BatchNormalization())

# Adding a third convolutional layer
model.add(Conv2D(64,(3,3),input_shape=input_shape))
model.add(Activation('relu'))
model.add(MaxPooling2D(pool_size=(2,2)))
model.add(BatchNormalization())
# Flattening
model.add(Flatten())

# Full connection
model.add(Dense(64, activation='relu',))
model.add(BatchNormalization())

model.add(Dense(1, activation='sigmoid'))

# Compiling the CNN
model.compile(loss='binary_crossentropy',optimizer='adam',metrics=['accuracy'])

2023-09-30 10:23:56.812950: I tensorflow/core/platform/cpu_feature_guard.cc:193] This TensorFlow binary is optimized with oneAPI Deep Neural Network Library (oneDNN) to use the following CPU instructions in performance-critical operations:  SSE4.1 SSE4.2
To enable them in other operations, rebuild TensorFlow with the appropriate compiler flags.


In [8]:
### Fitting the CNN to the images

print('starting training....')

model.fit(train_genarator,steps_per_epoch=train_genarator.samples//batch_size,
          epochs=10,
          validation_data=validation_genarator,
          validation_steps=validation_genarator.samples//batch_size)

starting training....
Epoch 1/10
47/47 [==============================] - 6s 111ms/step - loss: 0.8736 - accuracy: 0.6444 - val_loss: 0.6215 - val_accuracy: 0.7778
Epoch 2/10
47/47 [==============================] - 5s 105ms/step - loss: 0.5620 - accuracy: 0.7166 - val_loss: 0.8208 - val_accuracy: 0.5602
Epoch 3/10
47/47 [==============================] - 5s 106ms/step - loss: 0.5099 - accuracy: 0.7437 - val_loss: 1.2408 - val_accuracy: 0.5648
Epoch 4/10
47/47 [==============================] - 5s 105ms/step - loss: 0.4979 - accuracy: 0.7563 - val_loss: 1.6060 - val_accuracy: 0.5741
Epoch 5/10
47/47 [==============================] - 5s 106ms/step - loss: 0.4842 - accuracy: 0.7455 - val_loss: 1.6554 - val_accuracy: 0.5694
Epoch 6/10
47/47 [==============================] - 5s 106ms/step - loss: 0.4815 - accuracy: 0.7617 - val_loss: 1.1282 - val_accuracy: 0.5602
Epoch 7/10
47/47 [==============================] - 5s 105ms/step - loss: 0.4754 - accuracy: 0.7617 - val_loss: 2.5213 - val_a

In [9]:
model.evaluate(test_genarator)

4/4 [==============================] - 1s 119ms/step - loss: 0.4977 - accuracy: 0.8141


[0.4977009892463684, 0.8140703439712524]

In [10]:
print('training finished!!')

training finished!!


In [11]:
def obtain_inference(img):
    im = image.img_to_array(img)
    img1 = im/255
    img1 = img1.reshape((1,150,150,3))
    prediction = model.predict(img1)
    print(prediction)
    if prediction > 0.5:
        print('Good Pizza')
    else:
        print('Burnt Pizza')

In [13]:
img1 = image.load_img(r'data/pizza_ds/Test/Good/good (35).jpg',target_size=(150,150))
obtain_inference(img1)

1/1 [==============================] - 0s 106ms/step
[[0.82201415]]
Good Pizza


In [14]:
img2 = image.load_img(r'data/pizza_ds/Test/Burnt/12.jpg',target_size=(150,150))
obtain_inference(img2)

1/1 [==============================] - 0s 28ms/step
[[0.5833607]]
Good Pizza


### About Batch Normalization

    1.Every Batch of data from a layer is normalized with mean=0 and std=1 before its fed into subsequent layers. Every neuron has an adder & activation, so o/p of neurons u can standardize that with mean=0 & std=1 across a batch in a particular layer.
    2. The mean and standard deviation (or variance) are calculated for the 32 samples for each neuron individually, not for all neurons together. So if you have N neurons, you'll end up with N different means and N different variances (one for each neuron) based on the outputs of the 32 samples in the mini-batch.
    3.This ensures faster convergence and speeds up the training process